In [1]:
import mimicfouretl.bigquery_utils as bq

In [2]:
bq.set_credentials_file('../bq_credentials/client_secret.json')
# bq.set_project_id('mimic-iv-418015')
bq.set_project_id('micro-vine-412020')

In [3]:
client = bq.get_client(use_service_account_auth=True)
tables = bq.list_tables(client, 'micro-vine-412020.mimic_iv')
# tables = bq.list_tables(client, 'physionet-data.mimiciv_hosp')
tables

None


['micro-vine-412020.mimic_iv.hosp_admissions',
 'micro-vine-412020.mimic_iv.hosp_d_hcpcs',
 'micro-vine-412020.mimic_iv.hosp_d_icd_diagnoses',
 'micro-vine-412020.mimic_iv.hosp_d_icd_procedures',
 'micro-vine-412020.mimic_iv.hosp_d_labitems',
 'micro-vine-412020.mimic_iv.hosp_diagnoses_icd',
 'micro-vine-412020.mimic_iv.hosp_drgcodes',
 'micro-vine-412020.mimic_iv.hosp_emar',
 'micro-vine-412020.mimic_iv.hosp_emar_detail',
 'micro-vine-412020.mimic_iv.hosp_hcpcsevents',
 'micro-vine-412020.mimic_iv.hosp_labevents',
 'micro-vine-412020.mimic_iv.hosp_microbiologyevents',
 'micro-vine-412020.mimic_iv.hosp_omr',
 'micro-vine-412020.mimic_iv.hosp_patients',
 'micro-vine-412020.mimic_iv.hosp_pharmacy',
 'micro-vine-412020.mimic_iv.hosp_poe',
 'micro-vine-412020.mimic_iv.hosp_poe_detail',
 'micro-vine-412020.mimic_iv.hosp_prescriptions',
 'micro-vine-412020.mimic_iv.hosp_procedures_icd',
 'micro-vine-412020.mimic_iv.hosp_provider',
 'micro-vine-412020.mimic_iv.hosp_services',
 'micro-vine-412

In [4]:
spark = bq.get_spark_session(client)

24/03/23 17:55:50 WARN Utils: Your hostname, Dunyas-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.0.0.117 instead (on interface en0)
24/03/23 17:55:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/dunyaoguz/.ivy2/cache
The jars for the packages stored in: /Users/dunyaoguz/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-69f315cc-9cf8-485e-a05f-b5789d7d4f82;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/dunyaoguz/anaconda3/envs/mimic-iv-etl/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.37.0 in central
	[0.37.0] com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;latest.version
:: resolution report :: resolve 1173ms :: artifacts dl 2ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.37.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   1   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-69f315cc-9cf8-485e-a05f-b5789d7d4f82
	confs: [default]
	0 artifacts copied, 1 already retrieved (0kB/3ms)
24/03/23 17:55:51 WARN NativeCodeLoader: Unable to load native-hadoo

In [5]:
# results = bq.run_query(spark, "SELECT * FROM `physionet-data.mimiciv_hosp.admissions` LIMIT 10")
results = bq.run_query(spark, "SELECT * FROM `micro-vine-412020.mimic_iv.hosp_admissions` LIMIT 10")

OSError: Project was not passed and could not be determined from the environment.

In [ ]:
bq.display_sampled_df(results)